# Compare SED for different ages at different t_obs or redshifts but not in future

- author : Sylvie Dagoret-Campagne
- creation date : 2023/10/05
- last update : 2023/12/07



| computer | processor | kernel              |    date     |
| --- | --- | --- | --- |
| CC       | CPU       | conda_jax0325_py310 | 2023-11-10  |
| macbookpro | CPU | conda_jaxcpu_dsps_py310 | 2023-11-10  | 


libraries 
=========

jax
---

- jaxlib-0.3.25+cuda11.cudnn82
- jaxopt
- optax
- corner
- arviz
- numpyro

sps
---

- fsps
- prospect
- dsps
- diffstar
- diffmah



(conda_jax0325_py310) 
`/pbs/throng/lsst/users/dagoret/desc/StellarPopulationSynthesis>pip list | grep` 

| lib | version |
|--- | --- | 
|jax  |                         0.4.20 |
|jaxlib |                       0.4.20 |
|jaxopt  |                      0.8.2 |


    
 

https://dsps.readthedocs.io/en/latest/dsps_quickstart.html

This demo notebook begins by downloading the DSPS default option for the SSP spectral library. These data are stored at this URL in a flat hdf5 file with column names as expected by the dsps.load_ssp_templates function, which we will demonstrate below.

When downloading and storing SSP libraries, you can optionally use the DSPS_DRN environment variable to specify the default location where DSPS will look for SSP libraries. But here we’ll just save the downloaded data to tempdata.h5, directly pass the filename to the data loader. The load_ssp_templates that we’ll use to load these SSPs is just a convenience function - all of the DSPS functions that we’ll demonstrate in this notebook accept plain arrays and floats as inputs, and so you can store your SSP data on disk in whatever format you like.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.cm as cmx
from matplotlib.backends.backend_pdf import PdfPages
import copy

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  12
plt.rcParams['font.size'] = 12

In [ ]:
import numpy as np

## Inspect the SSP data¶

In [ ]:
#! curl https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/ssp_data_fsps_v3.2_lgmet_age.h5 > tempdata.h5
#! curl https://portal.nersc.gov/project/hacc/aphearin/DSPS_data/ssp_data_continuum_fsps_v3.2_lgmet_age.h5 > tempdata_v2.h5

In [ ]:
from dsps import load_ssp_templates
#ssp_data = load_ssp_templates(fn='tempdata_v2.h5')
ssp_data = load_ssp_templates(fn='tempdata.h5')

print(ssp_data._fields)

print('\nssp_lgmet.shape = {}'.format(ssp_data.ssp_lgmet.shape))
print('ssp_lg_age_gyr.shape = {}'.format(ssp_data.ssp_lg_age_gyr.shape))
print('ssp_wave.shape = {}'.format(ssp_data.ssp_wave.shape))
print('ssp_flux.shape = {}'.format(ssp_data.ssp_flux.shape))

In [ ]:
from dsps import calc_rest_sed_sfh_table_lognormal_mdf
from dsps import calc_rest_sed_sfh_table_met_table
from dsps.cosmology import age_at_z, DEFAULT_COSMOLOGY

In [ ]:
ssp_data.ssp_lgmet

In [ ]:
ssp_data.ssp_lg_age_gyr

The returned ssp_data is a namedtuple storing 4 ndarrays for the age-metallicity grid of the SSP spectra. Galaxy SEDs are calculated via probability-weighted sums of these spectral templates. For a galaxy observed at some, we’ll calculate the restframe SED of two different models in the cells below:

    a galaxy with a tabulated star formation history (SFH), and metallicity Z distributed as a lognormal about some median Z, using the calc_rest_sed_sfh_table_lognormal_mdf function.

    a galaxy with SFH table and also tabulated history of metallicity (ZH), using the calc_rest_sed_sfh_table_met_table function.

In the cells below, we’ll randomly generate an SFH and ZH for a galaxy, and then plot the results.

## Explore L_SPS

In [ ]:
# two mutually exclusive flags to see if we can provide SFR for t> tobs
# both flags can be True of False at the same time, but no selection on tobs will be done
FLAG_CUT_BELOW_TOBS = True
FLAG_CUT_AFTER_TOBS = False
FLAG_SET_YLIM = True

In [ ]:
import numpy as np
# table of ages in 100 bins in linear scale from 50 millions to 13.8 billions years
gal_t_table = np.linspace(0.05, 13.8, 100) # age of the universe in Gyr

# must put a true SFR using package diffstar
gal_sfr_table = np.zeros(len(gal_t_table)) # SFR in Msun/yr

# metallicity
gal_lgmet = -1.0 # log10(Z)
gal_lgmet_scatter = 0.2 # lognormal scatter in the metallicity distribution function

gal_lgmet_table = np.zeros(len(gal_t_table)) + gal_lgmet

In [ ]:
NBSPECTRA = len(gal_t_table)
# wavelength bin colors
jet = plt.get_cmap('jet')
cmap_reversed = jet.reversed()
cNorm = colors.Normalize(vmin=0, vmax=NBSPECTRA)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cmap_reversed)
all_colors = scalarMap.to_rgba(np.arange(NBSPECTRA), alpha=1)

The SED calculating functions require you specify the time of the observation, t_obs, rather than the redshift, z_obs. We’ll use the age_at_z function in dsps.cosmology to calculate the relationship between these two quantities, assuming the default redshift of DSPS. You could also use this same function to compute gal_t_table in case your input SFH is tabulated as a function of redshift.

In [ ]:
# defining the list of redshits/tobs
#all_redshifts = [0., 0.1, 0.5, 1.0, 3., 5.]
all_redshifts = [0., 1.0]

In [ ]:
# empty containers to start
all_sedinfos_for_allredshifts = []
all_sedinrestframe_for_allredshifts = []
all_sedtitles_for_allredshifts = []
all_sedtobs = []


for redshift in all_redshifts:
    
    # info
    # =====
    z_obs = redshift
    t_obs = age_at_z(z_obs, *DEFAULT_COSMOLOGY) # age of the universe in Gyr at z_obs, we cant see for t> t_obs
    t_obs = t_obs[0] # age_at_z function returns an array, but SED functions accept a float for this argument
    
    if FLAG_CUT_BELOW_TOBS:
        the_title = f"z_obs = {z_obs:.2f}, t_obs = {t_obs:.2f} Gyr, log(Z) = {gal_lgmet}, SFR before tobs"
    elif FLAG_CUT_AFTER_TOBS:
        the_title = f"z_obs = {z_obs:.2f}, t_obs = {t_obs:.2f} Gyr, log(Z) = {gal_lgmet}, SFR after tobs"
    elif FLAG_CUT_AFTER_TOBS ==  FLAG_CUT_BELOW_TOBS:
        the_title = f"z_obs = {z_obs:.2f}, t_obs = {t_obs:.2f} Gyr, log(Z) = {gal_lgmet}, SFR before and after tobs"
    
    # save title and tobs for that redshift
    all_sedtitles_for_allredshifts.append(the_title)
    all_sedtobs.append(t_obs)
    
    # Compute spectra
    # ==============
    all_sed_infos  = []
    all_sfr = []

    # loop on all ages stored in the gal_t_table (some age are above the current age of the universe
    for idx,time in enumerate(gal_t_table):
              
            
        #gal_sfr_table = np.zeros(len(gal_t_table)) # SFR in Msun/yr
        # validate one dirac peak in the SFR table to have the spectrum of the star population born at that time and have this age (indec,time)
        gal_sfr_table[idx] = 1.0
    
        all_sfr.append(copy.deepcopy(gal_sfr_table))
    
        # get the SED for that star population at that age
        sed_info = calc_rest_sed_sfh_table_lognormal_mdf(
        gal_t_table, gal_sfr_table, gal_lgmet, gal_lgmet_scatter,
        ssp_data.ssp_lgmet, ssp_data.ssp_lg_age_gyr, ssp_data.ssp_flux, t_obs)
        all_sed_infos.append(sed_info) 
    
        sed_info2 = calc_rest_sed_sfh_table_met_table(
        gal_t_table, gal_sfr_table, gal_lgmet_table, gal_lgmet_scatter,
        ssp_data.ssp_lgmet, ssp_data.ssp_lg_age_gyr, ssp_data.ssp_flux, t_obs)
    
        # reset all population at this age (index,time)
        gal_sfr_table[idx] = 0.0
    
    # Save all SED infos for that redshift
    # Only take the first calculation in sed_info, not that one calculated in sed_info2
    all_sedinfos_for_allredshifts.append(copy.deepcopy(all_sed_infos))
    
    # Plot
    #=====
    fig, axs = plt.subplots(1, 2,figsize=(18,5))

    ax1=axs[0]
    for idx,time in enumerate(gal_t_table):
        ax1.plot(gal_t_table, all_sfr[idx],"-",color=all_colors[idx])
    ax1.set_xlabel("age of the universe in Gyr")
    ax1.set_ylabel("SFR")
    ax1.axvline(t_obs,color="k",lw=3)
    ax1.axvspan(t_obs,13.8,color="grey",alpha=0.5)

    ax2=axs[1]
    __=ax2.loglog()
    
    all_sedinrestframe = []
    for idx,time in enumerate(gal_t_table):
        if FLAG_CUT_BELOW_TOBS and time < t_obs:
            ax2.plot(ssp_data.ssp_wave, all_sed_infos[idx].rest_sed,"-",color=all_colors[idx],lw=0.5)
            all_sedinrestframe.append(copy.deepcopy(all_sed_infos[idx].rest_sed))
        elif FLAG_CUT_AFTER_TOBS and time > t_obs:
            ax2.plot(ssp_data.ssp_wave, all_sed_infos[idx].rest_sed,"-",color=all_colors[idx],lw=0.5)
            all_sedinrestframe.append(copy.deepcopy(all_sed_infos[idx].rest_sed))
        elif FLAG_CUT_BELOW_TOBS == FLAG_CUT_AFTER_TOBS:
            ax2.plot(ssp_data.ssp_wave, all_sed_infos[idx].rest_sed,"-",color=all_colors[idx],lw=0.5)
            all_sedinrestframe.append(copy.deepcopy(all_sed_infos[idx].rest_sed))
    
    # save all SEDs at all tobs for that redshift
    all_sedinrestframe_for_allredshifts.append(all_sedinrestframe)
    
    if FLAG_SET_YLIM:
        ax2.set_ylim(1e-12,1e-5)
    ax2.grid()
    ax2.set_xlabel("$\lambda (\\AA)$")
    ax2.set_ylabel("$L_\\nu(\lambda)$")
    
    
    plt.tight_layout()
    plt.suptitle(the_title,y=1.05,fontsize=20,fontweight="bold")
    

In [ ]:

def SaveFigSpectraInPdf(nbspecperpage=6):
    
    Nspec= len(all_redshifts) # also all_sedinfos_for_allredshifts
    
    Nbpages = np.ceil(Nspec/nbspecperpage)
    idx_inpage = nbspecperpage
    idx_pagenum = 0
    idx = 0
    
    all_composed_figs = []
    
    # loop on redshifts
    for index in range(Nspec): 
        
        idx = index
        t_obs = all_sedtobs[idx]
        
        # get index on spectrum
        textstr = all_sedtitles_for_allredshifts[idx]
        #all_sed_inrestframe = all_sedinrestframe_for_allredshifts[idx]
        all_sedinfos = all_sedinfos_for_allredshifts[idx]
   
        # if we need to start a new page
        if idx_inpage == nbspecperpage:
            idx_inpage = 1  # start at the first row in the page from 1 to nbspecperpage
            idx_pagenum += 1  # increase page counter
            
            # this trigger the saving of the whole figure (including all spectra of the page)
            if idx_pagenum > 1:
                plt.show()
                all_composed_figs.append(fig)
            
            # this create a new page of figure 
            fig=plt.figure(figsize=(18,20))
            
        
            
        # this is not a new page, need to increase the row number inside this page from 1 to nbspecperpage    
        else:
            idx_inpage += 1   
        
        # index of the figure in the page
        idx_fig = 2*(idx_inpage-1)+1
        
        print(f"\t \t  SaveFigSpectraInPdf >>>>>>>>  spec num = {index}, pagenum = {idx_pagenum}/{Nbpages}, idx_inpage = {idx_inpage}, idx_fig = {idx_fig}")        
       
    
       
        
        # left image, two columns
        ax1 = fig.add_subplot(nbspecperpage, 2, idx_fig)
        # loop on all ages
        for idx1,time in enumerate(gal_t_table):
            ax1.plot(gal_t_table, all_sfr[idx1],"-",color=all_colors[idx1])
        ax1.set_xlabel("age of the universe in Gyr")
        ax1.set_ylabel("SFR")
        ax1.axvline(all_sedtobs[index],color="k",lw=3)
        ax1.axvspan(t_obs,13.8,color="grey",alpha=0.5)

    
        #right image, to columns
        
        # Get the spectrum info for that redshift         
        #all_sedinfos = all_sedinfos_for_allredshifts[index]
        label =  all_sedtitles_for_allredshifts[index]
        
        
        
        
        ax2 = fig.add_subplot(nbspecperpage, 2, idx_fig+1)
        
        __=ax2.loglog()
    
        # loop on ages
        for idx1,time in enumerate(gal_t_table):
                
            if FLAG_CUT_BELOW_TOBS and time < t_obs:
                ax2.plot(ssp_data.ssp_wave, all_sedinfos[idx1].rest_sed,"-",color=all_colors[idx1],lw=0.5)
            elif FLAG_CUT_AFTER_TOBS and time > t_obs:
                ax2.plot(ssp_data.ssp_wave, all_sedinfos[idx1].rest_sed,"-",color=all_colors[idx1],lw=0.5)
            elif FLAG_CUT_BELOW_TOBS  == FLAG_CUT_AFTER_TOBS:
                ax2.plot(ssp_data.ssp_wave, all_sedinfos[idx1].rest_sed,"-",color=all_colors[idx1],lw=0.5)
            
        if FLAG_SET_YLIM:
            ax2.set_ylim(1e-12,1e-5)
        
        ax2.grid()
        ax2.set_xlabel("$\lambda (\\AA)$")
        ax2.set_ylabel("$L_\\nu(\lambda)$")
        ax2.set_title(label)
        
             
    all_composed_figs.append(fig)                       
    return  all_composed_figs
    

In [ ]:
all_figs = SaveFigSpectraInPdf(nbspecperpage=6)

In [ ]:
if FLAG_CUT_BELOW_TOBS and not FLAG_CUT_AFTER_TOBS:
    multipage_pdf_file = f'multipage_pdf_galaxyspectra_sfrbeforetobs.pdf'
elif FLAG_CUT_AFTER_TOBS and not FLAG_CUT_BELOW_TOBS:
    multipage_pdf_file = f'multipage_pdf_galaxyspectra_sfraftertobs.pdf'
elif FLAG_CUT_BELOW_TOBS == FLAG_CUT_AFTER_TOBS:
    multipage_pdf_file = f'multipage_pdf_galaxyspectra_sfrbeforeandaftertobs.pdf'


In [ ]:
# gerate a multipage pdf:
with PdfPages(multipage_pdf_file) as pdf:
    for fig in all_figs:
        pdf.savefig(fig)
        plt.close()

In [ ]:
all_figs